In [17]:
import wikipedia
import wikipediaapi

Art and culture
Geography and places
Health and fitness
History and events
Mathematics and abstractions
Natural sciences and nature
People and self
Philosophy and thinking
Religion and spirituality
Social sciences and society
Technology and applied sciences

In [18]:
# drink_list = wikipedia.search("drink", results=1000)
# food_list = wikipedia.search("food", results=1000)
# ancient_list = wikipedia.search("ancient", results=1000)
# transportation_list = wikipedia.search("transportation", results=1000)
# shows_list = wikipedia.search("shows", results=1000)
# music_list = wikipedia.search("music", results=1000)
# actors_list = wikipedia.search("actors", results=1000)
# movies_list = wikipedia.search("movies", results=1000)
# finance_list = wikipedia.search("finance", results=1000)
# #
# language_list = wikipedia.search("language", results=1000)
# nature_list = wikipedia.search("nature", results=1000)
# environment_list = wikipedia.search("environment", results=1000)
# history_list = wikipedia.search("history", results=1000)
# econ_list = wikipedia.search("economics", results=1000)
# nature_list = wikipedia.search("nature", results=1000)
# lit_list = wikipedia.search("literature", results=1000)
# psych_list = wikipedia.search("psychology", results=1000)
# behavioral = wikipedia.search("behavioral", results=1000)
# global_list = wikipedia.search("global", results=1000)
# cities_list = wikipedia.search("cities", results=1000)
# anthropology = wikipedia.search("anthropology", results=1000)
# #
# art_culture = wikipedia.search("art culture", results=1000)
# geo_places = wikipedia.search("geography places", results=1000)
# health_fit = wikipedia.search("health fitness", results=1000)
# history_events = wikipedia.search("history events", results=1000)
# math = wikipedia.search("mathematics abstractions", results=1000)
# natural_science = wikipedia.search("natural sciences nature", results=1000)
# people_self = wikipedia.search("people self", results=1000)
# philosophy = wikipedia.search("philosophy thinking", results=1000)
# religion = wikipedia.search("religion spirituality", results=1000)
# social_society = wikipedia.search("social sciences society", results=1000)
# tech = wikipedia.search("technology applied sciences", results=1000)

In [19]:
# all_articles = (art_culture + geo_places + health_fit + history_events + 
#                 math + natural_science + people_self + philosophy + religion
#                 + social_society + tech + anthropology + cities_list + global_list
#                 + behavioral + psych_list + lit_list + nature_list + econ_list 
#                 + history_list + environment_list + nature_list + language_list
#                 + finance_list + movies_list + actors_list + music_list + shows_list
#                 + transportation_list + ancient_list + food_list + drink_list)

In [20]:
# all_articles_unique = list(set(all_articles))

In [21]:
# import pickle
# with open('data/unique_articles.pkl', 'wb') as f:
#     pickle.dump(all_articles_unique, f)

In [22]:
def collect_articles():
    docs2_long_text = []
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI) 
    for i in range(len(all_articles_unique)):
        temp = wiki_wiki.page(all_articles_unique[i])
        if len(temp.text) > 300:
            docs2_long_text.append(temp.text)
        else: 
            continue
    return docs2_long_text

In [27]:
# all_collected_articles = collect_articles()

In [23]:
# import pickle
# with open('data/collected_articles.pkl', 'wb') as f:
#     pickle.dump(all_collected_articles, f)

In [28]:
with open(r"data/collected_articles.pkl", "rb") as input_file:
    collected_articles = pickle.load(input_file)

In [29]:
import pandas as pd
df_corpus2 = pd.DataFrame()

In [30]:
df_corpus2['content'] = pd.Series(collected_articles)

In [31]:
! pip install textstat

In [32]:
import textstat

In [33]:
complexity = [textstat.flesch_kincaid_grade(doc) for doc in collected_articles]

In [34]:
df_corpus2['score'] = pd.Series(complexity)

In [35]:
import pickle
with open('data/df_corpus2.pkl', 'wb') as f:
    pickle.dump(df_corpus2, f)

## Load Files and Vectorize 

In [36]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [47]:
with open(r"data/df_corpus2.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [48]:
vec = TfidfVectorizer(stop_words='english', max_features=10)

In [49]:
vec = vec.fit(df_corpus2['content'])

In [50]:
#vec.vocabulary_

In [51]:
corpus2_vectors = vec.transform(df_corpus2['content']).toarray()

In [52]:
user_doc = "A watermelon is a type of edible fruit, they are 92% water. About 6% of a watermelon is sugar, which makes it very sweet. There are many different types of watermelon. Some have a green rind on the outside and a red-pink flesh on the inside, with brown seeds. Some can have yellow flesh, and some can be seedless. The green rind on the outside is not usually eaten, though it can be used as a vegetable. It can also be stewed or pickled. Most watermelons are oblong or spherical. In Japan, watermelons are grown in different shapes. Many people like to eat watermelon in the summer because the fruit is cool and refreshing. Watermelons are a great source of vitamin A, vitamin C, vitamin B6 and vitamin B1. They also contain potassium, magnesium, carotenoid antioxidant, and lycopene. Watermelons are fruits that come from a vine-like plant."

In [53]:
user_doc_vector = vec.transform([user_doc]).toarray()

In [54]:
user_doc_vector

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.72840959, 0.        , 0.        , 0.68514193, 0.        ]])

In [55]:
distances = cdist(user_doc_vector,
                  corpus2_vectors,
                  metric='cosine')[0]

In [56]:
distances.shape

(14216,)

In [57]:
ranking = np.argsort(distances)

In [58]:
ranking.shape

(14216,)

In [59]:
top_ten = ranking[:10]
top_ten

array([ 3879,  7216,  8986,  1943, 11400, 10636,  9629,  8609,  5384,
        1488])

In [60]:
distances[top_ten]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [61]:
df_corpus2.loc[ranking, 'content']

3879     With a climate as varied and extreme as India,...
7216     International finance (also referred to as int...
8986     A pakudos is a visual motif used by the Mangya...
1943     Construct (psychology), also hypothetical cons...
11400    The epicanthic fold is the skin fold of the up...
10636    Chuan (Chinese: 串, Dungan: Чўан, pinyin: chuàn...
9629     Global Village Tech Park is a software technol...
8609     Tejuíno is a cold beverage made from fermented...
5384     Gens was used by Lewis H. Morgan (in Ancient S...
1488     "Natural Science" is a song by the Canadian ro...
10581    Argument to moderation (Latin: argumentum ad t...
8306     Burgess may refer to:\n\nPeople\nBurgess (name...
11978    Trust for Nature is a not-for-profit organisat...
7376     Kochi is a city in the Indian state of Kerala....
7953     A self-directed Individual Retirement Account ...
6197     Vets Prevail, is a veteran-created online ment...
13135    Humans of some cultures eat octopus. The arms .

## Second Model 

In [110]:
import textstat
import operator
import wikipedia
import wikipediaapi

def give_harder_level(text):
    """
    Takes in text and returns a harder level read
    """
    ##find all the articles that are easier than this article 
    all_harder_text = []
    input_score = textstat.flesch_kincaid_grade(text)
    for i in range(len(df_corpus2['score'])):
        if df_corpus2['score'][i] > input_score:
            all_harder_text.append(df_corpus2['content'][i])
    
    vec = TfidfVectorizer(stop_words='english', max_features=2000)
    vec = vec.fit(all_harder_text)
    corpus2_vectors = vec.transform(all_harder_text).toarray()
    user_doc = text
    user_doc_vector = vec.transform([user_doc]).toarray() 
    user_doc_vector_short = np.argsort(user_doc_vector)[-25:]
    distances = cdist(user_doc_vector_short,
                  corpus2_vectors[-25:],
                  metric='cosine')[0]
    ranking = np.argsort(distances)
    top = ranking[0]
    best_match = df_corpus2['content'][top]
        
    print(distances[top])
    return (best_match)

In [113]:
sample = "fruit"

In [114]:
give_harder_level(sample)

0.6837722769838963


'Social conservatism is the belief that society is built upon a fragile network of relationships which need to be upheld through duty, traditional values and established institutions. This can include moral issues. Social conservatism is generally skeptical of social change, and believes in maintaining the status quo concerning social issues such as family life, sexual relations, and patriotism.Social conservatism encompasses a range of what may be thought of as reactionary positions on social issues. It developed as a reaction to what was perceived as dangerous tendencies within the liberal movements toward political radicalism and a wholesale rejection of "traditional values". In North America, since the mid to late 20th century, social conservatism arose as a response to federal action on social issues—such as LGBT rights and abortion—which members perceived as a threat to conservative values. Social conservatives also value the influence of religion in the public square, thus suppo